In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import talib
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('kline_1610026260_1618666439.csv').drop(columns=['Unnamed: 0','open_time','close_time','ignore'])

In [3]:
df

,open,high,low,close,volume,quote_asset_volume,num_trades,buy_volume,quote_volume
0,38282.81,38282.81,38201.01,38259.73,108.611866,4.152802e+06,2463.0,32.689829,1.249860e+06
1,38259.72,38320.17,38252.77,38304.32,77.440389,2.964820e+06,1606.0,31.735588,1.215102e+06
2,38304.32,38312.90,38238.35,38243.82,65.671076,2.513683e+06,1439.0,24.090259,9.220926e+05
3,38243.82,38279.68,38230.00,38279.12,59.950645,2.293327e+06,1105.0,26.408491,1.010132e+06
4,38279.12,38300.00,38250.02,38261.70,61.547511,2.355739e+06,1353.0,27.666768,1.058974e+06
...,...,...,...,...,...,...,...,...,...
143829,60750.36,60780.25,60691.10,60780.25,45.479245,2.761782e+06,2246.0,16.272949,9.883043e+05
143830,60780.24,60884.14,60773.05,60884.13,70.615901,4.295369e+06,2788.0,42.158266,2.564283e+06
143831,60884.13,60952.10,60884.13,60950.13,31.335621,1.909096e+06,2165.0,18.695376,1.138948e+06
143832,60950.12,60988.85,60903.97,60986.02,54.816537,3.341146e+06,2265.0,29.318344,1.786948e+06


In [4]:
close = df['close'].copy()
volume = df['volume'].copy()

In [5]:
# Setup TALIB indicators
TIMEPERIODS = [5, 8, 14, 30]
indicator_dict = {}
for timeperiod in TIMEPERIODS:
    indicator_dict[f'upperband_{timeperiod}'],indicator_dict[f'middleband_{timeperiod}'],indicator_dict[f'lowerband_{timeperiod}'] = talib.BBANDS(close,timeperiod=timeperiod)
    indicator_dict[f'dema_{timeperiod}'] = talib.DEMA(close,timeperiod)
    indicator_dict[f'ema_{timeperiod}'] = talib.EMA(close,timeperiod)
    indicator_dict[f'ma_{timeperiod}'] = talib.MA(close,timeperiod)
    
    # momentums
    indicator_dict[f'momentum_{timeperiod}'] = talib.MOM(close, timeperiod)
    indicator_dict[f'rsi_{timeperiod}'] = talib.RSI(close, timeperiod)
    
indicator_dict['ppo'] = talib.PPO(close)
indicator_dict['macd'], indicator_dict['macdsignal'], indicator_dict['macdhist'] = talib.MACD(close)
indicator_dict['obv'] = talib.OBV(close,volume)



In [ ]:
indicator_df = pd.DataFrame(indicator_dict)
indicator_df

In [7]:
print(indicator_df.shape)

(143834, 37)


### setup targets
use close prices of T+1 to T+5 as target?  
can consider using high/lows.  
Add these as extra columns  
model can predict multivariate or just a mean. not sure.

In [8]:
final_df = df.join(indicator_df)

In [14]:
for i in range(1,6):
    final_df[f'close_t{i}'] = df['close'].shift(-i)

final_df.dropna(inplace=True)


In [17]:
processed_np = np.array(final_df)

In [21]:
np.save('processed.npy',processed_np)